### Load Packages

In [2]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

## Load dataset and clean data - Adam

In [3]:
data = pd.read_csv('Life Expectancy Data.csv')

data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

gdp_per_capita = data.gdp/data.population
data['gdp_per_capita'] = gdp_per_capita

np.random.seed(2)
train = data.sample(round(data.shape[0]*0.8)) # 80% training, 20% test
test = data.drop(train.index)

/var/folders/0x/61vcmsg94cs94n1kc433lq8w0000gn/T/ipykernel_76720/1437335773.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/var/folders/0x/61vcmsg94cs94n1kc433lq8w0000gn/T/ipykernel_76720/1437335773.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


In [5]:
train.head(1)

country  year      status  life_expectancy  adult_mortality  \
1576  Malawi  2009  Developing             51.5            491.0   

      infant_deaths  alcohol  percentage_expenditure  hepatitis_b  measles  \
1576             36     1.16                7.915071         93.0       21   

      ...  total_expenditure  diphtheria  hiv/aids        gdp  population  \
1576  ...               1.18        93.0      14.9  42.737966   1471462.0   

      thinness__1-19_years  thinness_5-9_years  \
1576                   6.9                 6.8   

      income_composition_of_resources  schooling  gdp_per_capita  
1576                            0.415        9.9        0.000029  

[1 rows x 23 columns]

## Variable Selection/VIF - Judd

In [ ]:
# Get VIF values
print(train.corr())

In [ ]:


#Predictors VIF will be caluclated for:

X = train[['mpg', 'year', 'mileage', 'engineSize']]

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# add constant column
X = add_constant(X)
# create empty dataframe to input VIF values
vif_data=pd.DataFrame()
# create row for each predictor in X
vif_data['predictor'] = X.columns

# for loop to fill in VIF values 
for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values,i)
    
print(vif_data)

In [ ]:
#Function to find the best predictor out of p-k predictors and add it to the model containing the k predictors
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['Rsquared'].argmax()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
def forward_selection():
    models_best = pd.DataFrame(columns=["Rsquared", "model"])

    tic = time.time()
    predictors = []

    for i in range(1,len(X.columns)+1):    
        models_best.loc[i] = forward(predictors)
        predictors = list(models_best.loc[i]["model"].params.index[1:])

    toc = time.time()
    print("Total elapsed time:", (toc-tic), "seconds.")
    return models_best

In [ ]:
best_sub_plots()

In [ ]:
best_fwd_reg_model = models_best['model'][8]
best_fwd_reg_model.summary()

## Final Regression Analysis - Alissa